In [2]:
!pip install tensorflow tensorflow-federated

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [40]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(30,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [41]:
def client_update(model, dataset, epochs=1, patience=3):
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='loss', patience=patience, restore_best_weights=True)
    history = model.fit(dataset[0], dataset[1], epochs=epochs, verbose=2, callbacks=[early_stopping])
    accuracy = history.history['accuracy'][-1]
    return model.get_weights(), accuracy

In [42]:
def federated_averaging(client_weights):
    for i, weights in enumerate(client_weights):
        print(f"Client {i} weights shape: {[w.shape for w in weights]}")

    # here we will average the weights
    average_weights = [np.mean(np.array(weights), axis=0) for weights in zip(*client_weights)]

    # Print the shape of the averaged weights for global model
    print(f"Averaged weights shape: {[w.shape for w in average_weights]}")

    return average_weights

In [43]:
def federated_learning(num_rounds, num_clients, client_data, epochs=1, patience=3):
    global_model = create_model()

    for round in range(num_rounds):
        client_weights = []
        client_accuracies = []

        for client_id in range(num_clients):
            model = create_model()
            model.set_weights(global_model.get_weights())

            print(f"Client {client_id} initial weights shape: {[w.shape for w in model.get_weights()]}")

            weights, accuracy = client_update(model, client_data[client_id], epochs=epochs, patience=patience)
            client_weights.append(weights)
            client_accuracies.append(accuracy)

        global_weights = federated_averaging(client_weights)
        global_model.set_weights(global_weights)

        print(f"Round {round + 1} global weights shape: {[w.shape for w in global_model.get_weights()]}")
        print(f"Client accuracies for round {round + 1}: {client_accuracies}")

    return global_model


In [44]:
# here i have and example with breast cancer dataset
# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_client_data(X, y, num_clients):
    client_data = []
    data_per_client = len(X) // num_clients
    for i in range(num_clients):
        client_X = X[i * data_per_client:(i + 1) * data_per_client]
        client_y = y[i * data_per_client:(i + 1) * data_per_client]
        client_data.append((client_X, client_y))
    return client_data

num_clients = 10
client_data = create_client_data(X_train, y_train, num_clients)

if __name__ == "__main__":
    num_rounds = 5
    epochs = 10
    patience = 3
    final_model = federated_learning(num_rounds, num_clients, client_data, epochs=epochs, patience=patience)

    # Evaluate the model on test data
    final_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    test_loss, test_acc = final_model.evaluate(X_test, y_test, verbose=2)
    print(f"Test accuracy: {test_acc}")

Client 0 initial weights shape: [(30, 16), (16,), (16, 1), (1,)]
Epoch 1/10
2/2 - 1s - loss: 1.0739 - accuracy: 0.3333 - 508ms/epoch - 254ms/step
Epoch 2/10
2/2 - 0s - loss: 0.9826 - accuracy: 0.4889 - 13ms/epoch - 7ms/step
Epoch 3/10
2/2 - 0s - loss: 0.9007 - accuracy: 0.5111 - 11ms/epoch - 5ms/step
Epoch 4/10
2/2 - 0s - loss: 0.8199 - accuracy: 0.5556 - 9ms/epoch - 5ms/step
Epoch 5/10
2/2 - 0s - loss: 0.7575 - accuracy: 0.5778 - 9ms/epoch - 5ms/step
Epoch 6/10
2/2 - 0s - loss: 0.7029 - accuracy: 0.6000 - 14ms/epoch - 7ms/step
Epoch 7/10
2/2 - 0s - loss: 0.6571 - accuracy: 0.6222 - 11ms/epoch - 5ms/step
Epoch 8/10
2/2 - 0s - loss: 0.6151 - accuracy: 0.6889 - 10ms/epoch - 5ms/step
Epoch 9/10
2/2 - 0s - loss: 0.5802 - accuracy: 0.6889 - 10ms/epoch - 5ms/step
Epoch 10/10
2/2 - 0s - loss: 0.5509 - accuracy: 0.7111 - 10ms/epoch - 5ms/step
Client 1 initial weights shape: [(30, 16), (16,), (16, 1), (1,)]
Epoch 1/10
2/2 - 1s - loss: 0.9105 - accuracy: 0.3111 - 515ms/epoch - 258ms/step
Epoch 2